In [35]:
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import datetime
import random
import time
import sys
import os

In [2]:
np.set_printoptions(precision=6)
tf.random.set_random_seed(10)
np.random.seed(10)

## Define data-set const

In [3]:
mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()

ncolor = 1
nclass = 10
width  = 28
height = 28
ntrain = 55000
nval = 5000
ntest  = len(x_test)
total_train_size = len(x_train)

x_train = x_train/255.0
x_train = np.reshape(x_train, (total_train_size, -1))
x_test = x_test/255.0
x_test = np.reshape(x_test, (ntest, -1))

labels = np.zeros((total_train_size, nclass))
labels[np.arange(total_train_size), y_train] = 1
y_train = labels

x_val = x_train[ntrain:total_train_size]
y_val = y_train[ntrain:total_train_size]

x_train = x_train[0:ntrain]
y_train = y_train[0:ntrain]

labels = np.zeros((ntest, nclass))
labels[np.arange(ntest), y_test] = 1
y_test = labels

print("Number of Training Data: ", len(x_train), len(y_train))
print("Number of Validation Data: ", len(x_val), len(y_val))
print("Number of Test Data: ", len(x_test), len(y_test))
print("Number of class: ", nclass)

Number of Training Data:  55000 55000
Number of Validation Data:  5000 5000
Number of Test Data:  10000 10000
Number of class:  10


In [4]:
class dqn_model(object):
    def __init__(self, n_class, lr=1e-3, gamma = 0.9, scopeName = "dqn"):
        with tf.variable_scope(scopeName):
            # Placeholder Input
            s = tf.placeholder(tf.float32, [None, n_class])
            b = tf.placeholder(tf.float32, [None, 1])
            t = tf.placeholder(tf.float32, [None, 1])
            R = tf.placeholder(tf.float32, [None, 1])
            avg_V_ = tf.placeholder(tf.float32, [None, 1])

            # Network
            concat = tf.concat([s, b, t], 1)
            fc1 = tf.layers.dense(concat, 128, tf.nn.relu)
            fc2 = tf.layers.dense(fc1, 10, tf.nn.relu)
            V = tf.layers.dense(fc2, 1, tf.nn.relu)
            avg_V = tf.reduce_mean(V)

            # Optimizer
            td_error = R + gamma * avg_V_ - avg_V
            loss = tf.square(td_error)
            train_op = tf.train.AdamOptimizer(lr).minimize(loss)

            # Variables
            self.s = s
            self.b = b
            self.t = t
            self.R = R
            self.V = V
            self.loss = loss
            self.avg_V = avg_V
            self.avg_V_ = avg_V_
            self.td_error = td_error
            self.train_op = train_op

In [5]:
class classification_model(object):
    def __init__(self, img_w, img_h, n_class, lr=1e-4, scopeName = "cnn"):
        with tf.variable_scope(scopeName):
            # Placeholder Input
            x = tf.placeholder(tf.float32, [None, img_w*img_h])           # (batch, height, width, channel)
            y_ = tf.placeholder(tf.float32, [None, n_class])            # input y
            img = tf.reshape(x, [-1, img_w, img_h, 1])

            # Network
            conv1 = tf.layers.conv2d(img, 32, 5, 1, 'same', activation=tf.nn.relu)
            pool1 = tf.layers.max_pooling2d(conv1, pool_size=2, strides=2,)
            conv2 = tf.layers.conv2d(pool1, 64, 5, 1, 'same', activation=tf.nn.relu)
            pool2 = tf.layers.max_pooling2d(conv2, 2, 2)
            pool2f = tf.reshape(pool2, [-1, 7*7*64])
            fc1 = tf.layers.dense(pool2f, 1024, tf.nn.relu)
            output = tf.layers.dense(fc1, n_class)
            y = tf.contrib.layers.softmax(output)

            # Optimizer
            loss = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))
            correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
            accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32), name='accuracy')
            optimizer = tf.train.AdamOptimizer(lr)
            train_op = optimizer.minimize(loss)

            # Variables
            self.x = x
            self.y = y
            self.y_ = y_
            self.loss = loss
            self.accuracy = accuracy
            self.train_op = train_op
            self.optimizer = optimizer

In [6]:
sess =tf.Session()
cnn_init = classification_model(width, height, nclass, scopeName="cnn_init")
cnn = classification_model(width, height, nclass, scopeName="cnn_train")
dqn = dqn_model(nclass, scopeName="dqn")

sess.run(tf.global_variables_initializer())
tf_vars = tf.trainable_variables()
cnn_init_var = [var for var in tf_vars if 'cnn_init' in var.name]
cnn_var = [var for var in tf_vars if 'cnn_train' in var.name]
dqn_var = [var for var in tf_vars if 'dqn' in var.name]
reset_optimizer_op = tf.variables_initializer(cnn.optimizer.variables())

## Define variables

In [30]:
total_steps = 1000
total_budget = 2000

c_BatchSize = 10
d_BatchSize = 50
c_Epoch = 30


now = datetime.datetime.now()
filename = str(now) + "_steps_" + str(total_steps) + "_budgets_" + str(total_budget) + "_batchsize_" + str(c_BatchSize) + "_d_BatchSize_" + str(d_BatchSize)
print(filename)

2019-04-30 10:47:25.336690_steps_1000_budgets_2000_batchsize_10_d_BatchSize_50


20190430_104725_s1000_b2000_cbs10_dbs50_e30


In [8]:
bgn_time = time.time()
for step in range(total_steps):

    # Re-init cnn_var with values of cnn_init_var
    for idx, var in enumerate(cnn_init_var):
        sess.run(tf.assign(cnn_var[idx], var))
    sess.run(reset_optimizer_op)

    R = sess.run(cnn.accuracy, {cnn.x:x_val, cnn.y_:y_val})
    print("\nSTEP:", step, " TIME TAKEN:", time.time()-bgn_time, " val_acc:", R)
    bgn_time = time.time()

    # Reset Variables
    log_batch = np.array([])

    for c_step in range(int(total_budget/c_BatchSize)):
        trained = c_step * c_BatchSize
        remain_budget_percentage = (total_budget-trained) / total_budget
        future_remain_budget_percentage = (total_budget-trained-c_BatchSize) / total_budget
        remain_steps_percentage = (total_steps-step) / total_steps

        # Get Classification Probability
        start = c_step * d_BatchSize
        end = (c_step + 1) * d_BatchSize
        imgs = x_train[start:end]
        lbls = y_train[start:end]
        cnn_y = sess.run(cnn.y, {cnn.x:imgs, cnn.y_:lbls})

        # Predict the Accurarcy Score
        remain_budget = np.full((d_BatchSize, 1), remain_budget_percentage, dtype=float)
        remain_steps = np.full((d_BatchSize, 1), remain_steps_percentage, dtype=float)
        feed_dict = {dqn.s:cnn_y, dqn.b:remain_budget, dqn.t:remain_steps}
        predict_acc = sess.run(dqn.V, feed_dict)

        # Select the best [c_BatchSize]-th images
        ranked = np.argsort(predict_acc.squeeze())
        top_idx = ranked[-c_BatchSize:]
        low_idx = ranked[:-c_BatchSize]
        batch_x = x_train[start + top_idx[-c_BatchSize:]]
        batch_y = y_train[start + top_idx[-c_BatchSize:]]
        top_predict = predict_acc[top_idx]
        top_cnn_y = cnn_y[top_idx]

        # Train Classification Network
        for i in range(c_Epoch):
            sess.run(cnn.train_op,{cnn.x:batch_x, cnn.y_:batch_y})

        #Train DQN Network
        R = 100 * sess.run(cnn.accuracy, {cnn.x:x_val, cnn.y_:y_val})
        S = cnn_y[top_idx]
        S_ = sess.run(cnn.y, {cnn.x:batch_x, cnn.y_:batch_y})
        avg_V_ = sess.run(dqn.avg_V,
                      {dqn.s: S_,
                       dqn.b: np.full((c_BatchSize, 1), future_remain_budget_percentage),
                       dqn.t: np.full((c_BatchSize, 1), remain_steps_percentage)})

        sess.run(dqn.train_op,
                 {dqn.s: S,
                  dqn.b: np.full((c_BatchSize, 1), remain_budget_percentage),
                  dqn.t: np.full((c_BatchSize, 1), remain_steps_percentage),
                  dqn.R: np.full((c_BatchSize, 1), R),
                  dqn.avg_V_: np.full((c_BatchSize, 1), avg_V_)})
        print("Iter:", c_step, "val_acc:", R, "top predict:", top_predict.mean(), "low predict:", predict_acc[low_idx].mean())

        log = {"dqn_step": step,
               "cnn_step": c_step,
               "val_acc": R,
               "top_predict_mean": top_predict.mean(),
               "low_predict": predict_acc[low_idx].mean(),
               "top_predict": top_predict.squeeze(),
               "low_predict": predict_acc[low_idx].squeeze()
              }
        log_batch = np.append(log_batch, log)

    # write data to files 
    f = open(filename+".txt","a")
    for log in log_batch:
        f.write(str(log) + '\n')
    f.close()


STEP: 0  TIME TAKEN: 1.1423766613006592  val_acc: 0.135
Iter: 0 val_acc: 18.780000507831573 top predict: 0.011398765 low predict: 0.010983972
Iter: 1 val_acc: 19.859999418258667 top predict: 0.026712378 low predict: 0.019524986
Iter: 2 val_acc: 15.65999984741211 top predict: 0.044031754 low predict: 0.023760248
Iter: 3 val_acc: 34.67999994754791 top predict: 0.054650772 low predict: 0.044566415
Iter: 4 val_acc: 39.46000039577484 top predict: 0.096566536 low predict: 0.07466476
Iter: 5 val_acc: 46.86000049114227 top predict: 0.0970177 low predict: 0.07568939
Iter: 6 val_acc: 47.3800003528595 top predict: 0.13211437 low predict: 0.10320939
Iter: 7 val_acc: 38.13999891281128 top predict: 0.16143864 low predict: 0.11640452
Iter: 8 val_acc: 51.16000175476074 top predict: 0.18003634 low predict: 0.118761756
Iter: 9 val_acc: 58.160001039505005 top predict: 0.16099402 low predict: 0.10866928
Iter: 10 val_acc: 54.00000214576721 top predict: 0.19921826 low predict: 0.15461603
Iter: 11 val_acc: 

Iter: 100 val_acc: 82.59999752044678 top predict: 7.694945 low predict: 6.610827
Iter: 101 val_acc: 82.74000287055969 top predict: 8.137789 low predict: 6.8162065
Iter: 102 val_acc: 82.8000009059906 top predict: 8.430296 low predict: 7.1175356
Iter: 103 val_acc: 82.85999894142151 top predict: 8.741998 low predict: 7.424266
Iter: 104 val_acc: 85.64000129699707 top predict: 8.841253 low predict: 7.35
Iter: 105 val_acc: 85.10000109672546 top predict: 9.140746 low predict: 7.57384
Iter: 106 val_acc: 84.60000157356262 top predict: 9.404856 low predict: 7.7744493
Iter: 107 val_acc: 88.20000290870667 top predict: 9.585573 low predict: 8.091876
Iter: 108 val_acc: 87.84000277519226 top predict: 10.014329 low predict: 8.341834
Iter: 109 val_acc: 87.77999877929688 top predict: 9.758947 low predict: 8.271643
Iter: 110 val_acc: 87.91999816894531 top predict: 10.347139 low predict: 8.682283
Iter: 111 val_acc: 89.21999931335449 top predict: 10.000954 low predict: 8.610181
Iter: 112 val_acc: 88.880002


STEP: 1  TIME TAKEN: 220.04747581481934  val_acc: 0.135
Iter: 0 val_acc: 18.960000574588776 top predict: 64.955215 low predict: 64.89101
Iter: 1 val_acc: 31.560000777244568 top predict: 68.3884 low predict: 65.49014
Iter: 2 val_acc: 45.159998536109924 top predict: 68.28136 low predict: 65.633804
Iter: 3 val_acc: 32.85999894142151 top predict: 70.56779 low predict: 66.82705
Iter: 4 val_acc: 48.96000027656555 top predict: 74.99918 low predict: 68.312645
Iter: 5 val_acc: 30.180001258850098 top predict: 76.79404 low predict: 70.09709
Iter: 6 val_acc: 46.799999475479126 top predict: 79.32232 low predict: 75.293106
Iter: 7 val_acc: 38.199999928474426 top predict: 78.965706 low predict: 72.7491
Iter: 8 val_acc: 40.860000252723694 top predict: 80.38347 low predict: 76.0296
Iter: 9 val_acc: 45.3000009059906 top predict: 79.77611 low predict: 72.77444
Iter: 10 val_acc: 60.64000129699707 top predict: 78.798775 low predict: 73.16024
Iter: 11 val_acc: 40.43999910354614 top predict: 77.439896 low p

Iter: 101 val_acc: 84.74000096321106 top predict: 222.62358 low predict: 192.37991
Iter: 102 val_acc: 83.96000266075134 top predict: 228.50166 low predict: 197.19086
Iter: 103 val_acc: 82.89999961853027 top predict: 230.5676 low predict: 200.68439
Iter: 104 val_acc: 77.0799994468689 top predict: 226.98672 low predict: 198.01494
Iter: 105 val_acc: 76.99999809265137 top predict: 233.24692 low predict: 201.25847
Iter: 106 val_acc: 76.48000121116638 top predict: 232.65703 low predict: 200.3621
Iter: 107 val_acc: 78.06000113487244 top predict: 236.80359 low predict: 204.10327
Iter: 108 val_acc: 78.20000052452087 top predict: 242.15092 low predict: 207.86899
Iter: 109 val_acc: 78.29999923706055 top predict: 229.51921 low predict: 206.54619
Iter: 110 val_acc: 78.88000011444092 top predict: 241.50656 low predict: 209.48209
Iter: 111 val_acc: 80.51999807357788 top predict: 227.36267 low predict: 208.47116
Iter: 112 val_acc: 83.25999975204468 top predict: 250.97836 low predict: 216.35068
Iter: 1

Iter: 0 val_acc: 16.279999911785126 top predict: 526.7301 low predict: 526.2462
Iter: 1 val_acc: 37.97999918460846 top predict: 545.9669 low predict: 522.85333
Iter: 2 val_acc: 40.380001068115234 top predict: 545.9521 low predict: 527.63983
Iter: 3 val_acc: 40.880000591278076 top predict: 547.8156 low predict: 520.00964
Iter: 4 val_acc: 33.05999934673309 top predict: 577.15393 low predict: 536.1331
Iter: 5 val_acc: 44.38000023365021 top predict: 598.66003 low predict: 564.1765
Iter: 6 val_acc: 45.60000002384186 top predict: 589.289 low predict: 538.9362
Iter: 7 val_acc: 32.339999079704285 top predict: 597.30493 low predict: 538.7648
Iter: 8 val_acc: 41.94000065326691 top predict: 596.1734 low predict: 562.91376
Iter: 9 val_acc: 52.34000086784363 top predict: 573.99225 low predict: 516.53845
Iter: 10 val_acc: 49.300000071525574 top predict: 556.24634 low predict: 515.7344
Iter: 11 val_acc: 43.16000044345856 top predict: 567.67523 low predict: 506.70004
Iter: 12 val_acc: 46.2000012397766

...

Iter: 71 val_acc: 49.61999952793121 top predict: 581.06775 low predict: 322.16138
Iter: 72 val_acc: 54.420000314712524 top predict: 570.3054 low predict: 215.04346
Iter: 73 val_acc: 51.80000066757202 top predict: 556.1154 low predict: 335.83502
Iter: 74 val_acc: 53.359997272491455 top predict: 583.4025 low predict: 376.2204
Iter: 75 val_acc: 64.70000147819519 top predict: 570.3085 low predict: 334.7372
Iter: 76 val_acc: 64.42000269889832 top predict: 584.8621 low predict: 378.52554
Iter: 77 val_acc: 64.44000005722046 top predict: 589.19244 low predict: 378.119
Iter: 78 val_acc: 70.77999711036682 top predict: 576.92377 low predict: 372.7891
Iter: 79 val_acc: 68.87999773025513 top predict: 591.6403 low predict: 382.0124
Iter: 80 val_acc: 67.1999990940094 top predict: 586.7586 low predict: 413.323
Iter: 81 val_acc: 64.38000202178955 top predict: 591.0503 low predict: 382.30084
Iter: 82 val_acc: 63.94000053405762 top predict: 594.5805 low predict: 402.35187
Iter: 83 val_acc: 64.89999890327

Iter: 172 val_acc: 67.25999712944031 top predict: 743.41565 low predict: 563.5235
Iter: 173 val_acc: 66.57999753952026 top predict: 726.19165 low predict: 533.3109
Iter: 174 val_acc: 67.58000254631042 top predict: 737.3712 low predict: 567.7462
Iter: 175 val_acc: 64.21999931335449 top predict: 718.094 low predict: 499.57773
Iter: 176 val_acc: 65.39999842643738 top predict: 741.43005 low predict: 543.9667
Iter: 177 val_acc: 65.52000045776367 top predict: 730.74347 low predict: 551.6237
Iter: 178 val_acc: 64.89999890327454 top predict: 730.5838 low predict: 556.5878
Iter: 179 val_acc: 64.819997549057 top predict: 741.646 low predict: 548.4232
Iter: 180 val_acc: 68.45999956130981 top predict: 739.2561 low predict: 543.66583
Iter: 181 val_acc: 73.36000204086304 top predict: 718.4208 low predict: 524.23615
Iter: 182 val_acc: 71.34000062942505 top predict: 740.79254 low predict: 505.49115
Iter: 183 val_acc: 70.93999981880188 top predict: 730.5824 low predict: 480.72372
Iter: 184 val_acc: 75.

Iter: 73 val_acc: 65.61999917030334 top predict: 556.03827 low predict: 434.93008
Iter: 74 val_acc: 60.93999743461609 top predict: 545.9731 low predict: 389.81226
Iter: 75 val_acc: 61.080002784729004 top predict: 531.9176 low predict: 331.22845
Iter: 76 val_acc: 53.89999747276306 top predict: 562.6911 low predict: 370.19354
Iter: 77 val_acc: 58.21999907493591 top predict: 572.7739 low predict: 436.1172
Iter: 78 val_acc: 56.40000104904175 top predict: 574.5172 low predict: 402.24774
Iter: 79 val_acc: 58.70000123977661 top predict: 561.0718 low predict: 310.8272
Iter: 80 val_acc: 62.04000115394592 top predict: 565.0751 low predict: 357.75885
Iter: 81 val_acc: 62.699997425079346 top predict: 575.85535 low predict: 296.4489
Iter: 82 val_acc: 66.33999943733215 top predict: 580.0999 low predict: 351.07486
Iter: 83 val_acc: 67.00000166893005 top predict: 578.6485 low predict: 354.18
Iter: 84 val_acc: 70.80000042915344 top predict: 563.7939 low predict: 329.07227
Iter: 85 val_acc: 69.919997453

Iter: 174 val_acc: 72.3800003528595 top predict: 754.0963 low predict: 552.91235
Iter: 175 val_acc: 72.02000021934509 top predict: 731.83923 low predict: 533.0803
Iter: 176 val_acc: 71.96000218391418 top predict: 757.6395 low predict: 559.88525
Iter: 177 val_acc: 73.82000088691711 top predict: 746.2088 low predict: 572.28424
Iter: 178 val_acc: 73.37999939918518 top predict: 755.36316 low predict: 596.60486
Iter: 179 val_acc: 73.72000217437744 top predict: 760.817 low predict: 603.7141
Iter: 180 val_acc: 70.44000029563904 top predict: 761.56433 low predict: 583.8878
Iter: 181 val_acc: 76.38000249862671 top predict: 741.82666 low predict: 563.4165
Iter: 182 val_acc: 80.59999942779541 top predict: 747.4977 low predict: 588.36365
Iter: 183 val_acc: 81.45999908447266 top predict: 752.10016 low predict: 604.90125
Iter: 184 val_acc: 81.04000091552734 top predict: 759.3221 low predict: 595.72546
Iter: 185 val_acc: 79.55999970436096 top predict: 765.7582 low predict: 630.4139
Iter: 186 val_acc:

KeyboardInterrupt: 